<div class="frontmatter text-center">
<h1> Introduction to Data Science and Programming</h1>
<h2>Class 24: Network models</h2>
<h3>IT University of Copenhagen, Fall 2019</h3>
<h3>Instructor: Michael Szell</h3>
</div>

# Source
This notebook was adapted from:
* Bruno Gonçalves / Data4Sci: https://github.com/DataForScience/Networks
* James Bagrow: http://bagrow.com/dsv/

In [ ]:
from pprint import pprint

import numpy as np
import matplotlib.pyplot as plt 

We import the previous Graph Class

In [ ]:
from Graph import *

In [ ]:
G = Graph()

In [ ]:
dir(G)

## Erdős-Rényi 

Network models are algorithms to generate synthetic network topologies. We start with Erdős-Rényi model where each pair of nodes is added with a fixed probability

In [ ]:
@add_method(Graph)
def erdos_renyi_graph(N, p):
    nodes = list(range(N))
    edges = []
    
    for i in range(N):
        for j in range(i+1, N):
            if np.random.random() < p:
                edges.append((i, j))
    
    G = Graph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)

    return G

Let's generate a relatively small ER graph

In [ ]:
ER = Graph.erdos_renyi_graph(10000, 0.01)

With these parameters, we expect the average degree to be $\langle k\rangle=Np=100$ and the distribution to be approximately Gaussian. We start by generating the degre distribution

In [ ]:
Pk = ER.degree_distribution()

And make a quick plot to verify it's shape

In [ ]:
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1])

axes.plot(Pk.T[0], Pk.T[1])
axes.set_title('ER network degree distribution')
axes.set_xlabel('k')
axes.set_ylabel('P(k)');

Finally, the average degree is:

$$\langle k \rangle=\sum_k k P(k)$$

In [ ]:
kavg = np.dot(Pk.T[0], Pk.T[1])
print(kavg)

## Regular Ring

We now proceed to defining a regular ring graph where each node is connected with K/2 neighbors on the left and K/2 neighbours on the right. This will be a stepping stone towards the full fledged Watts-Strogats model.

In [ ]:
@add_method(Graph)
def ring_graph(N, K):
    nodes = np.arange(N)
    edges = []
    
    K = K//2
    
    for i in range(N):
        for j in range(i+1, i+K+1):
            edges.append((i, j%N))
        
    G = Graph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)
    
    return G

To generate a small regular ring we do:

In [ ]:
RG = Graph.ring_graph(100, 4)

The degree distribution is simple: All nodes have degree 4

In [ ]:
Pk = RG.degree_distribution()

In [ ]:
Pk

## Watts-Strogatz model

The first step towards defining a WS model is to implementing the rewiring procedure. This takes an existing graph and randomly rewires its edges with probability $p$

In [ ]:
@add_method(Graph)
def _rewire(self, p):
    node_labels = list(self._nodes.keys())
    nodes = dict(zip(node_labels, range(len(node_labels))))
    
    edges = self.edgelist()
        
    for node_i, node_j, data in edges:
        if nodes[node_j] > nodes[node_i]: # Make sure we just rewire one end of the edge

            if np.random.random() < p:
                new_node_j = node_labels[np.random.randint(len(node_labels))]
                self._edges[node_i][new_node_j] = self._edges[node_i][node_j]
                self._edges[node_i].pop(node_j)

                self._edges[new_node_j][node_i] = self._edges[node_j][node_i]
                self._edges[node_j].pop(node_i)

So if we apply it to the previous graph, we significantly impact the degree distribution

In [ ]:
RG = Graph.ring_graph(10000, 4)
RG._rewire(0.3)
WS = RG
Pk = WS.degree_distribution()


Which now has a non-zero spread while still peaked at 4:

In [ ]:
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1])

axes.plot(Pk.T[0], Pk.T[1])
axes.set_title('Watts-Strogatz network degree distribution')
axes.set_xlabel('k')
axes.set_ylabel('P(k)');

The Watts-Strogatz model is then:

In [ ]:
@add_method(Graph)
def watts_strogatz_graph(N, K, p):
    G = Graph.ring_graph(N, K)
    G._rewire(p)
    
    return G

## Barabasi-Albert model

We also implement the Barabasi-Albert model for the case where we start with 3 fully connected nodes and add one edge at each time step:

In [ ]:
@add_method(Graph)
def barabasi_albert_graph(N):
    G = Graph()

    nodes = range(N)
    G.add_nodes_from(nodes)

    edges = [0,1,1,2,2,0]

    for node_i in range(3, N):
        pos = np.random.randint(len(edges))
        node_j = edges[pos]

        edges.append(node_i)
        edges.append(node_j)

    edges = zip(nodes, edges[1::2])

    G.add_edges_from(edges)

    return G

As expected, the degree distribution is heavy-tailed:

In [ ]:
BA = Graph.barabasi_albert_graph(100000)
Pk = BA.degree_distribution()

In [ ]:
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1])

axes.loglog(Pk.T[0], Pk.T[1])
axes.set_title('BA network degree distribution')
axes.set_xlabel('k')
axes.set_ylabel('P(k)');

# Random Walks and Paths

A path on a graph is a sequence of visited nodes. Here we implement a random walk over a graph to generate a sequence of randomly visited nodes. 

In [ ]:
@add_method(Graph)
def random_walk(self, source, steps):
    path = [source]
    
    for i in range(steps):
        source = path[-1]
        NN = [node for node in self._edges[source]]
        
        chosen = NN[np.random.randint(len(NN))]
        path.append(chosen)
        
    return path

We generate a path of 1,000,000 steps

In [ ]:
path = BA.random_walk(0, 1000000)

And count how many times each node is visited

In [ ]:
visits = Counter(path)
counts = list(visits.items())
counts.sort(key=lambda x:x[1], reverse=True)

The top 10 most visited nodes are:

In [ ]:
counts[:10]

For comparison, their degrees are:

In [ ]:
deg = BA.degrees()
top10 = [i for i, j in counts[:10]]

for i, node in enumerate(top10):
    print(node, deg[node], counts[i][1])

And we see that the number of visits is strongly correlated with the node degree. To make it clearer:

In [ ]:
data = []
for node in deg:
    data.append((deg[node], visits[node]))
data = np.array(data)

# Plot
fig = plt.figure(figsize=(6, 4))
axes = fig.add_axes([0, 0, 1, 1])

axes.loglog(data.T[0], data.T[1], 'o')
axes.set_title('BA network random walker visits')
axes.set_xlabel('Degree k')
axes.set_ylabel('Visits');

# Sampling Methods

We also consider sampling approaches. These are often used when we must interact with some system (call an API, perform interviews, etc) to extract information about the network structure.

We start with the random walk sampling. This is just a random walk on the graph as we saw above. 

In [ ]:
@add_method(Graph)
def random_walk_sample(self, source, maxsteps=1000):
    seen = []
    queue = source
    
    for _ in range(maxsteps+1):
        user_id = queue
        seen.append(user_id)

        NN = list(self.neighbors(user_id))
        
        # Randomly select one of the neighbors
        queue = NN[np.random.randint(len(NN))]

    return seen

On the other hand, snowball sampling returns a compact set of nodes that forms a denser connected component:
<img src="files/snowballsampling.png" width="400px"/>

In [ ]:
@add_method(Graph)
def snowball_sample(self, source, max_depth = 3):
    seen = set()
    queue = set()

    queue.add(source)
    queue2 = set()

    for _ in range(max_depth+1):
        while queue:
            user_id = queue.pop()
            seen.add(user_id)

            NN = self.neighbours(user_id)

            for node in NN:
                if node not in seen:
                    queue2.add(node)

        queue.update(queue2)
        queue2 = set()

    return seen

In the case of the WS model the regular structure of the graph is reflected by just a few nodes being returned:

In [ ]:
seen = WS.snowball_sample(0, 3)
len(seen)

For comparison, we can also see a big difference to the BA model by performing a snowball sample with similar parameters:

In [ ]:
seen = BA.snowball_sample(0)
len(seen)

Here we recover a substantial part of the graph. The reason for this is: in a graph with a heavy-tailed distribution we quickly reach one (or more) of the hubs that are connected to a significant portion of the total graph. 